In [55]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

<u>Read CSVs to Data Frame</u>

In [56]:
credit_record = "Resources/credit_record.csv"
credit_df = pd.read_csv(credit_record)
credit_df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [57]:
customer_app = "Resources/application_record.csv"
customer_df = pd.read_csv(customer_app)
customer_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


<u>Clean DataFrame AND merge data</u>

In [58]:
# 0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month
# Set X and C to 0, means they dont have pass due
credit_df["STATUS"] =  credit_df["STATUS"].replace(["X"],0)
credit_df["STATUS"] =  credit_df["STATUS"].replace(["C"],0)

# switch STATUS indo all numeric, in order to sort
credit_df['STATUS'] = credit_df["STATUS"].apply(pd.to_numeric)
credit_status_df = credit_df.groupby('ID')['STATUS'].max().reset_index()


# Save as a new csv 
credit_status_df.to_csv('Resources/credit_status_csv', index = False)

credit_status_df.head()



,ID,STATUS
0,5001711,0
1,5001712,0
2,5001713,0
3,5001714,0
4,5001715,0


In [59]:
credit_status_df.reset_index()
credit_status_df.head()

,ID,STATUS
0,5001711,0
1,5001712,0
2,5001713,0
3,5001714,0
4,5001715,0


In [60]:
new_customer_df = customer_df[["ID", "CODE_GENDER", 'DAYS_BIRTH', "AMT_INCOME_TOTAL", "NAME_FAMILY_STATUS", "DAYS_EMPLOYED"]].copy()

# Rename the columns title
clean_customer_df = new_customer_df.rename(columns={"ID": "Client ID", "CODE_GENDER": "Gender", 'DAYS_BIRTH': "Age", "AMT_INCOME_TOTAL": "Annual Income", "NAME_FAMILY_STATUS": "Marital Status", "DAYS_EMPLOYED": "Years of Employment"})


#turn the numbers into age
clean_customer_df["Age"] = round(clean_customer_df["Age"]/-365,0).astype(int)
clean_customer_df["Years of Employment"]= round(clean_customer_df["Years of Employment"]/-365,0).astype(int)

clean_customer_df.head()


,Client ID,Gender,Age,Annual Income,Marital Status,Years of Employment
0,5008804,M,33,427500.0,Civil marriage,12
1,5008805,M,33,427500.0,Civil marriage,12
2,5008806,M,59,112500.0,Married,3
3,5008808,F,52,270000.0,Single / not married,8
4,5008809,F,52,270000.0,Single / not married,8


In [61]:
clean_customer_df.shape

(438557, 6)

<u>Create connections </u>

In [62]:
#connect to local database (pgadmin)
connection = 'postgres:password@localhost:5432/ETL-Project'
engine = create_engine(f'postgresql://{connection}')

In [63]:
engine.table_names()

['Customer', 'Credit Record']

<u>Load DataFrames into database</u>

In [64]:
clean_customer_df.to_sql(name = "Customer", con = engine, if_exists = "append", index = False)

In [65]:
# Confirm data has been added by querying table
pd.read_sql_query('SELECT * FROM "Customer"', con = engine).head()

,Client ID,Gender,Age,Annual Income,Marital Status,Years of Employment
0,5008804,M,33,427500,Civil marriage,12
1,5008805,M,33,427500,Civil marriage,12
2,5008806,M,59,112500,Married,3
3,5008808,F,52,270000,Single / not married,8
4,5008809,F,52,270000,Single / not married,8


In [66]:
credit_status_df.to_sql(name = "Credit Record", con = engine, if_exists = "append", index = False)

In [67]:
# Confirm data has been added by querying table
pd.read_sql_query('SELECT * FROM "Credit Record"', con = engine).head()

,ID,STATUS
0,5001711,0
1,5001712,0
2,5001713,0
3,5001714,0
4,5001715,0
